In [30]:
import os
import sys
from datetime import datetime

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from numba import njit
import numpy as np
from lib.utils import create_windows, file_to_data_frame, ExtendedPortfolio
import pandas as pd
import vectorbt as vbt

In [31]:
# leemos el csv
(s_name, ohlcv) = file_to_data_frame(
    "/Users/pilo/development/itba/pf/Binance_Minute_OHLC_CSVs/shorts/Binance_BTCUSDT_minute_3000.csv")
# agarramos solo las columnas que necesitamos
cols = ohlcv.columns
print(cols)
#ohlcv.get(["Open", "High", "Low", "Close", "Volume"]).vbt.ohlcv.plot().show_png()
ohlc = ohlcv.get(["Open", "High", "Low", "Close"])
print(ohlc.head())

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Volume USDT', 'Tradecount'], dtype='object')
                         Open      High       Low     Close
date                                                       
2021-03-16 22:08:00  56669.99  56700.00  56666.32  56676.18
2021-03-16 22:09:00  56676.18  56713.70  56624.73  56699.99
2021-03-16 22:10:00  56699.99  56740.57  56680.69  56732.66
2021-03-16 22:11:00  56732.66  56739.83  56658.45  56700.00
2021-03-16 22:12:00  56700.01  56751.50  56700.00  56748.48


In [32]:
# creamos las ventanas
figure, windows = create_windows(ohlc=ohlc, n=8, window_len=0.6, training_set_len=0.4)
(in_df, in_df_index), (out_df, _) = windows
figure.show()

In [33]:
in_df.head()

split_idx         0                                       1            \
               Open      High       Low     Close      Open      High   
0          56669.99  56700.00  56666.32  56676.18  56546.71  56555.75   
1          56676.18  56713.70  56624.73  56699.99  56518.57  56523.99   
2          56699.99  56740.57  56680.69  56732.66  56481.65  56535.06   
3          56732.66  56739.83  56658.45  56700.00  56330.89  56363.02   
4          56700.01  56751.50  56700.00  56748.48  56341.79  56441.11   

split_idx                             2            ...         5            \
                Low     Close      Open      High  ...       Low     Close   
0          56483.30  56518.57  55814.39  55832.23  ...  54724.41  54824.95   
1          56466.10  56481.65  55787.37  55794.43  ...  54802.89  54861.83   
2          56323.24  56330.88  55736.13  55736.13  ...  54857.64  54957.88   
3          56286.31  56341.64  55680.57  55688.80  ...  54892.90  54954.62   
4          56341.79  56428.18  55688.79  55697.04  ...  54856.82  55013.44   

split_idx         6                                       7            \
               Open      High       Low     Close      Open      High   
0          55168.30  55168.30  55106.26  55146.17  56133.10  56159.41   
1          55146.16  55184.21  55058.74  55068.07  56148.82  56251.34   
2          55068.07  55148.39  55017.40  55143.99  56243.89  56350.00   
3          55143.99  55149.01  54969.37  54974.52  56349.99  56426.98   
4          54978.17  55149.98  54960.88  55110.78  56409.19  56600.00   

split_idx                      
                Low     Close  
0          56093.36  56148.82  
1          56148.82  56243.88  
2          56219.31  56350.00  
3          56300.00  56409.19  
4          56400.00  56599.99  

[5 rows x 32 columns]

In [34]:
portfolio_kwargs = dict(
    direction='longonly',
    freq='m'
)


In [35]:
# creamos el indicador
@njit
def apply_alpha_nb(open: np.ndarray, high: np.ndarray, low: np.ndarray, close: np.ndarray, buy_threshold: float,
                  sell_threshold: float):
    aux = (close - open) / (high - low + 0.001)
    aux = np.where(aux >= buy_threshold, 1, aux)
    aux = np.where(aux <= -sell_threshold, -1, aux)
    return aux

AlphaInd = vbt.IndicatorFactory(
    input_names=['open', 'high', 'low', 'close'],
    param_names=['buy_threshold', 'sell_threshold'],
    output_names=['signal']
).from_apply_func(apply_alpha_nb)
# dir(AlphaInd)

In [36]:
# Corremos con todas las combinaciones
def simulate_all_params(ohlc_windows, params_range):
    # creamos las señales
    open = ohlc_windows.xs("Open", level=1, axis=1)
    high = ohlc_windows.xs("High", level=1, axis=1)
    low = ohlc_windows.xs("Low", level=1, axis=1)
    close = ohlc_windows.xs("Close", level=1, axis=1)
    momentum = AlphaInd.run(open=open, high=high, low=low, close=close,
                            buy_threshold=params_range, sell_threshold=params_range,
                            param_product=True,
                            short_name="alpha")
    ones = np.full(momentum.signal.shape[-1], 1)
    entry_signal = momentum.signal_equal(ones)
    exit_signal = momentum.signal_equal(-ones)
    return ExtendedPortfolio.from_signals(close, entry_signal, exit_signal, **portfolio_kwargs)


# Optimizamos para el in y el out
params_range = np.linspace(0.4, 1, 10, endpoint=False)
in_elr = simulate_all_params(in_df, params_range).expected_log_returns()
#out_elr = simulate_all_params(out_df, params_range).expected_log_returns()

In [37]:
def get_best_index(performance, higher_better=True):
    if higher_better:
        return performance[performance.groupby('split_idx').idxmax()].index
    return performance[performance.groupby('split_idx').idxmin()].index

def get_best_params(best_index, level_name):
    return best_index.get_level_values(level_name).to_numpy()

In [38]:
#Buscamos el índice de los mejores resultados del in
in_best_index = get_best_index(in_elr)
print(in_best_index)

MultiIndex([(0.97,  0.1, 0),
            (0.97, 0.88, 1),
            (0.94, 0.95, 2),
            (0.97, 0.97, 3),
            (0.94, 0.97, 4),
            (0.95, 0.97, 5),
            (0.97, 0.97, 6),
            (0.95, 0.99, 7)],
           names=['alpha_buy_threshold', 'alpha_sell_threshold', 'split_idx'])


In [39]:
in_best_buy_thresholds = get_best_params(in_best_index, 'alpha_buy_threshold')
in_best_sell_thresholds = get_best_params(in_best_index, 'alpha_sell_threshold')
in_best_threshold_pairs = np.array(list(zip(in_best_buy_thresholds, -in_best_sell_thresholds)))

print(in_best_threshold_pairs)

[[ 0.97 -0.1 ]
 [ 0.97 -0.88]
 [ 0.94 -0.95]
 [ 0.97 -0.97]
 [ 0.94 -0.97]
 [ 0.95 -0.97]
 [ 0.97 -0.97]
 [ 0.95 -0.99]]


In [40]:
pd.DataFrame(in_best_threshold_pairs, columns=['buy_threshold', 'sell_threshold']).vbt.plot().show()

In [41]:
# Corremos el out con los mejores parámetros de in
#close, high, low, open = list(map(lambda tu: tu[1], in_df.groupby(level=1, axis=1)))
open = in_df.xs("Open", level=1, axis=1)
high = in_df.xs("High", level=1, axis=1)
low = in_df.xs("Low", level=1, axis=1)
close = in_df.xs("Close", level=1, axis=1)
momentum = AlphaInd.run(open=open, high=high, low=low, close=close,
                        buy_threshold=in_best_buy_thresholds, sell_threshold=in_best_sell_thresholds,
                        short_name="alpha", per_column=True)
ones = np.full(momentum.signal.shape, 1)
entry_signal = momentum.signal_equal(ones, crossover=True)
exit_signal = momentum.signal_equal(-ones, crossover=True)
# imprimo para confirmar que haya algún true
entry_signal.loc[entry_signal[in_best_buy_thresholds[0], in_best_sell_thresholds[0], 0]==True].head()

alpha_buy_threshold   0.97          0.94   0.97   0.94   0.95   0.97   0.95
alpha_sell_threshold  0.10   0.88   0.95   0.97   0.97   0.97   0.97   0.99
split_idx                0      1      2      3      4      5      6      7
65                    True  False  False  False  False  False  False  False
69                    True  False  False  False  False  False  False  False
110                   True  False  False  False  False  False  False  False
142                   True  False  False  False  False  False  False  False
169                   True  False  False  False  False  False  False  False

In [42]:
trade_price = close
out_test_port = ExtendedPortfolio.from_signals(trade_price, entry_signal, exit_signal, **portfolio_kwargs)
out_test_elr = out_test_port.expected_log_returns()
print(out_test_elr)

alpha_buy_threshold  alpha_sell_threshold  split_idx
0.97                 0.10                  0            0.000783
                     0.88                  1            0.002778
0.94                 0.95                  2            0.004884
0.97                 0.97                  3            0.011202
0.94                 0.97                  4            0.008017
0.95                 0.97                  5            0.008951
0.97                 0.97                  6            0.007368
0.95                 0.99                  7            0.003215
Name: expected_log_returns, dtype: float64


In [43]:
# simulamos Buy&Hold de cada in y out window y tomamos el expected log returns (elr)
close_columns = list(filter(lambda col: "Close" in col[1], in_df.columns))
in_hold_port = ExtendedPortfolio.from_holding(in_df[close_columns], **portfolio_kwargs)
out_hold_port = ExtendedPortfolio.from_holding(out_df[close_columns], **portfolio_kwargs)
print(in_hold_port.trades.values[:4])
print(out_hold_port.trades.values[:4])
in_hold_elr = in_hold_port.expected_log_returns()
out_hold_elr = out_hold_port.expected_log_returns()
print(in_hold_elr, out_hold_elr)

[(0, 0, 0.00176441, 0, 56676.18, 0., 1079, 55085.25, 0., -2.80705227, -0.02807052, 0, 0, 0)
 (1, 1, 0.00176933, 0, 56518.57, 0., 1079, 57320.95, 0.,  1.41967498,  0.01419675, 0, 0, 1)
 (2, 2, 0.00179252, 0, 55787.37, 0., 1079, 58310.  , 0.,  4.52186579,  0.04521866, 0, 0, 2)
 (3, 3, 0.00178371, 0, 56062.98, 0., 1079, 58933.58, 0.,  5.12031291,  0.05120313, 0, 0, 3)]
[(0, 0, 0.00181437, 0, 55115.66, 0., 718, 58825.2 , 0.,  6.73046463,  0.06730465, 0, 0, 0)
 (1, 1, 0.00173685, 0, 57575.51, 0., 718, 58969.7 , 0.,  2.42149831,  0.02421498, 0, 0, 1)
 (2, 2, 0.00171616, 0, 58269.66, 0., 718, 58368.46, 0.,  0.16955651,  0.00169557, 0, 0, 2)
 (3, 3, 0.00169218, 0, 59095.51, 0., 718, 57752.64, 0., -2.2723723 , -0.02272372, 0, 0, 3)]
split_idx       
0          Close   -0.028472
1          Close    0.014097
2          Close    0.044226
3          Close    0.049935
4          Close    0.059499
5          Close    0.069589
6          Close    0.051498
7          Close    0.029813
Name: expected_lo

In [44]:
cv_results_df = pd.DataFrame({
    'in_sample_hold': in_hold_elr.values,
    #'in_sample_median': in_elr.groupby('split_idx').median().values,
    'in_sample_best': in_elr[in_best_index].values,
    'out_sample_hold': out_hold_elr.values,
    #'out_sample_median': out_elr.groupby('split_idx').median().values,
    'out_sample_test': out_test_elr.values
})

cv_results_df.vbt.plot(
    trace_kwargs=[
        dict(line_color=vbt.settings.color_schema['blue']),
        dict(line_color=vbt.settings.color_schema['blue'], line_dash='dash'),
        dict(line_color=vbt.settings.color_schema['blue'], line_dash='dot'),
        dict(line_color=vbt.settings.color_schema['orange']),
        dict(line_color=vbt.settings.color_schema['orange'], line_dash='dash'),
        dict(line_color=vbt.settings.color_schema['orange'], line_dash='dot')
    ]
).show()

In [45]:
for col in in_best_index:
    out_test_port.trades.plot(column=col).show()
out_test_port.trades.plot_pnl(column=col).show()



datetime.datetime(2021, 4, 9, 17, 59, 35, 753261)